In [1]:
import sys
import os

# Add the parent directory of 'vegetation-period-NDVI' to the sys.path
sys.path.append(os.path.abspath(os.path.join('..')))


In [2]:
import ee
import geemap
from time_series import extract_time_ranges, get_harmonic_ts

In [3]:
ee.Initialize(project="thurgau-irrigation")

In [4]:
yr = "2018"
# TODO: it return a List of computed objects but it should be an ee.List of ee.List of ee.Dictionary 
time_intervals = extract_time_ranges(
    time_range=[yr + "-03-01", yr + "-10-31"], agg_interval=15
)
print(time_intervals.getInfo())

[[{'type': 'Date', 'value': 1519862400000}, {'type': 'Date', 'value': 1521201600000}], [{'type': 'Date', 'value': 1521201600000}, {'type': 'Date', 'value': 1522540800000}], [{'type': 'Date', 'value': 1522540800000}, {'type': 'Date', 'value': 1523836800000}], [{'type': 'Date', 'value': 1523836800000}, {'type': 'Date', 'value': 1525132800000}], [{'type': 'Date', 'value': 1525132800000}, {'type': 'Date', 'value': 1526472000000}], [{'type': 'Date', 'value': 1526472000000}, {'type': 'Date', 'value': 1527811200000}], [{'type': 'Date', 'value': 1527811200000}, {'type': 'Date', 'value': 1529107200000}], [{'type': 'Date', 'value': 1529107200000}, {'type': 'Date', 'value': 1530403200000}], [{'type': 'Date', 'value': 1530403200000}, {'type': 'Date', 'value': 1531742400000}], [{'type': 'Date', 'value': 1531742400000}, {'type': 'Date', 'value': 1533081600000}], [{'type': 'Date', 'value': 1533081600000}, {'type': 'Date', 'value': 1534420800000}], [{'type': 'Date', 'value': 1534420800000}, {'type': '

In [5]:
type(time_intervals)

ee.ee_list.List

In [6]:
type(ee.List(time_intervals).get(0))

ee.computedobject.ComputedObject

In [7]:
test_aoi2 = ee.Geometry.Polygon(
    [
        [
            [8.737412756413452, 47.648802254332814],
            [8.737412756413452, 47.61548626028342],
            [8.829423254460327, 47.61548626028342],
            [8.829423254460327, 47.648802254332814],
        ]
    ]
)

In [8]:
filtered_sentinel_data = get_harmonic_ts(year=int(yr), aoi=test_aoi2, time_intervals=time_intervals)

In [9]:
type(
    filtered_sentinel_data
)

ee.imagecollection.ImageCollection

In [10]:
filtered_sentinel_data.first().bandNames().getInfo()

['NDVI', 't', 'constant', 'cos', 'sin', 'cos2', 'sin2', 'fitted']

In [11]:
Map = geemap.Map()

# Define visualization parameters
vis_params = {
    'bands': ['NDVI'],
    'min': 0,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

# Add the ee.ImageCollection to the map
Map.addLayer(filtered_sentinel_data.first(), vis_params, 'Filtered Sentinel Data')

# Display the map
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…